In [1]:
import pandas as pd
import numpy as np
import re
import os
import utils 
import string

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [2]:
data_dir = "data/"

files = ["H-1B_Disclosure_Data_FY16.xlsx", 
            "H-1B_Disclosure_Data_FY15_Q4.xlsx", 
            "H-1B_FY14_Q4.xlsx", 
            "LCA_FY2013.xlsx"]



In [3]:
data = pd.read_excel(data_dir + files[1])

In [4]:
columns = list(data)
print(columns)
data.columns = ['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_ADDRESS1', 'EMPLOYER_ADDRESS2', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAIC_CODE', 'TOTAL_WORKERS', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_WAGE_SOURCE', 'PW_WAGE_SOURCE_YEAR', 'PW_WAGE_SOURCE_OTHER', 'WAGE_RATE_OF_PAY', 'WAGE_UNIT_OF_PAY', 'H-1B_DEPENDENT', 'WILLFUL VIOLATOR', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE']
print(data.dtypes)

['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_ADDRESS1', 'EMPLOYER_ADDRESS2', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAIC_CODE', 'TOTAL WORKERS', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_WAGE_SOURCE', 'PW_WAGE_SOURCE_YEAR', 'PW_WAGE_SOURCE_OTHER', 'WAGE_RATE_OF_PAY', 'WAGE_UNIT_OF_PAY', 'H-1B_DEPENDENT', 'WILLFUL VIOLATOR', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE']
CASE_NUMBER                      object
CASE_STATUS                      object
CASE_SUBMITTED           datetime64[ns]
DECISION_DATE            datetime64[ns]
VISA_CLASS                       object
EMPLOYMENT_START_DATE            object

In [5]:
data.head(5)

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS1,EMPLOYER_ADDRESS2,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,PW_WAGE_SOURCE,PW_WAGE_SOURCE_YEAR,PW_WAGE_SOURCE_OTHER,WAGE_RATE_OF_PAY,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE
0,I-200-09121-701936,WITHDRAWN,2015-02-05,2015-02-05,H-1B,02/09/2015,02/28/2015,"MEDTRONIC, INC.",710 MEDTRONIC PARKWAY NE,NaN,MINNEAPOLIS,MN,55432,UNITED STATES OF AMERICA,NaN,7.635053e+09,NaN,DEBRA SCHNEIDER,MINNEAPOLIS,MN,MECHANICAL ENGINEER,17-2141.00,MECHANICAL ENGINEERS,334510.0,1.0,Y,19000.0,Year,Level I,OES,2014.0,ONLINE DATA SURVEY,20000 -,Year,N,N,EDEN PRAIRIE,STERNS,CA,55412
1,I-200-09146-796321,CERTIFIED,2014-12-12,2014-12-18,H-1B,01/05/2015,01/04/2018,UNIVERSITY OF OKLAHOMA,905 ASP AVE,NEL BLDG 207,NORMAN,OK,73019,UNITED STATES OF AMERICA,NaN,4.053252e+09,NaN,NaN,NaN,NaN,ASSISTANT PROFESSOR,25-1032,"ENGINEERING TEACHERS, POSTSECONDARY",611310.0,1.0,Y,42860.0,Year,Level I,Other,2014.0,OFLC ONLINE DATA CENTER,85000.00 -,Year,N,N,NORMAN,CLEVELAND,OK,73019
2,I-200-09180-329758,WITHDRAWN,2013-01-03,2014-12-15,H-1B,01/07/2013,01/06/2016,NOKIA INC.,6021 CONNECTION DRIVE,NaN,IRVING,TX,75039,UNITED STATES OF AMERICA,NaN,9.728945e+09,NaN,HASEENA ENU,DALLAS,TX,RELEASE AND SOFTWARE TEST MANAGER,15-1799,"COMPUTER OCCUPATIONS, ALL OTHER*",517212.0,1.0,Y,73965.0,Year,Level II,OES,2012.0,OFLC ONLINE DATA CENTER,94000 -,Year,N,N,SAN DIEGO,SAN DIEGO,CA,92127
3,I-200-09183-259985,CERTIFIED,2015-03-10,2015-03-16,H-1B,09/07/2015,09/07/2018,"OMRON OILFIELD AND MARINE, INC.",9510 N. HOUSTON ROSSLYN ROAD,NaN,HOUSTON,TX,77088,UNITED STATES OF AMERICA,NaN,7.138492e+09,NaN,ELDON KAKUDA,SCHAUMBURG,IL,CONTROL ENGINEERING SPECIALIST,17-2072,"ELECTRONICS ENGINEERS, EXCEPT COMPUTER",335314.0,2.0,Y,65998.0,Year,Level I,OES,2014.0,OFLC ONLINE DATA CENTER,66000 - 70000,Year,N,N,HOUSTON,HARRIS,TX,77088
4,I-200-09189-800933,CERTIFIED,2014-12-17,2014-12-23,H-1B,06/06/2015,06/05/2018,FEDERAL HOME LOAN MORTGAGE CO.,8250 JONES BRANCH DRIVE,MSA3E,MCLEAN,VA,22102,UNITED STATES OF AMERICA,NaN,7.039189e+09,NaN,JAMES ALEXANDER,WASHINGTON,DC,DEVELOPMENT SENIOR,15-1131,COMPUTER PROGRAMMERS,522294.0,1.0,Y,96907.0,Year,Level IV,OES,2014.0,OFLC ONLINE DATA CENTER,97000 - 143300,Year,N,N,MCLEAN,FAIRFAX,VA,22102


In [6]:
print(len(data))

618804


In [7]:
print(len(data.CASE_NUMBER.unique()))

618671


In [8]:
count = data.groupby('CASE_NUMBER').count()

In [9]:
data.loc[data['CASE_NUMBER'].isin(count[count['CASE_STATUS'] > 1].index)]

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS1,EMPLOYER_ADDRESS2,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,PW_WAGE_SOURCE,PW_WAGE_SOURCE_YEAR,PW_WAGE_SOURCE_OTHER,WAGE_RATE_OF_PAY,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE
164108,I-200-15037-239766,WITHDRAWN,2015-02-06,2015-02-11,H-1B,08/03/2015,08/01/2018,INFOSYS LIMITED,6100 TENNYSON PARKWAY,SUITE 200,PLANO,TX,75024,UNITED STATES OF AMERICA,NaN,4.694673e+09,NaN,NaN,NaN,NaN,TECHNOLOGY ANALYST - US,15-1121,COMPUTER SYSTEMS ANALYSTS,541511.0,1.0,Y,47923.00,Year,Level I,OES,2014.0,OFLC ONLINE DATA CENTER,67445 -,Year,Y,N,MIAMISBURG,MONTGOMERY,OH,45342
164109,I-200-15037-239766,CERTIFIED,2015-02-06,2015-02-17,H-1B,08/03/2015,08/01/2018,INFOSYS LIMITED,6100 TENNYSON PARKWAY,SUITE 200,PLANO,TX,75024,UNITED STATES OF AMERICA,NaN,4.694673e+09,NaN,NaN,NaN,NaN,TECHNOLOGY LEAD - US,15-1121,COMPUTER SYSTEMS ANALYSTS,541511.0,1.0,Y,62941.00,Year,Level II,OES,2014.0,OFLC ONLINE DATA CENTER,71047 -,Year,Y,N,MIAMISBURG,MONTGOMERY,OH,45342
164415,I-200-15037-299899,WITHDRAWN,2015-02-08,2015-02-11,H-1B,08/04/2015,08/02/2018,INFOSYS LIMITED,6100 TENNYSON PARKWAY,SUITE 200,PLANO,TX,75024,UNITED STATES OF AMERICA,NaN,4.694673e+09,NaN,NaN,NaN,NaN,ASSOCIATE CONSULTANT - US,15-1121,COMPUTER SYSTEMS ANALYSTS,541511.0,1.0,Y,62899.00,Year,Level I,OES,2014.0,OFLC ONLINE DATA CENTER,63272 -,Year,Y,N,HILLSBORO,WASHINGTON,OR,97124
164416,I-200-15037-299899,CERTIFIED,2015-02-08,2015-02-17,H-1B,08/04/2015,08/02/2018,INFOSYS LIMITED,6100 TENNYSON PARKWAY,SUITE 200,PLANO,TX,75024,UNITED STATES OF AMERICA,NaN,4.694673e+09,NaN,NaN,NaN,NaN,ASSOCIATE CONSULTANT - US,15-1121,COMPUTER SYSTEMS ANALYSTS,541511.0,1.0,Y,62899.00,Year,Level I,OES,2014.0,OFLC ONLINE DATA CENTER,63272 -,Year,Y,N,HILLSBORO,WASHINGTON,OR,97124
167228,I-200-15037-909445,CERTIFIED,2015-02-24,2015-03-05,H-1B,08/12/2015,08/11/2018,TRADEWEB MARKETS LLC,1177 AVENUE OF THE AMERICAS,NaN,NEW YORK,NY,10036,UNITED STATES OF AMERICA,NaN,6.464306e+09,NaN,MARIAH WILSON,BOSTON,MA,"AVP, IT, DEVELOPER",15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",541512.0,1.0,Y,73445.00,Year,Level I,OES,2014.0,OFLC ONLINE DATE CENTER,80000 - 80000,Year,N,N,JERSEY CITY,HUDSON,NJ,07311
167229,I-200-15037-909445,CERTIFIED,2015-02-24,2015-03-05,H-1B,08/12/2015,08/11/2018,"ACTIMIZE, INC.",1359 BROADWAY,5TH FLOOR,NEW YORK,NY,10018,UNITED STATES OF AMERICA,NaN,2.129944e+09,NaN,GEORGE LESTER,BOSTON,MA,SOFTWARE DEVELOPER,15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",541511.0,1.0,Y,57949.00,Year,Level I,OES,2014.0,OFLC ONLINE DATA CENTER,85000 - 85000,Year,Y,N,ALPHARETTA,FULTON,GA,30009
167258,I-200-15037-914751,WITHDRAWN,2015-02-07,2015-02-11,H-1B,08/02/2015,07/31/2018,INFOSYS LIMITED,6100 TENNYSON PARKWAY,SUITE 200,PLANO,TX,75024,UNITED STATES OF AMERICA,NaN,4.694673e+09,NaN,NaN,NaN,NaN,SYSTEMS ENGINEER - US,15-1131,COMPUTER PROGRAMMERS,541511.0,1.0,Y,55245.00,Year,Level I,OES,2014.0,OFLC ONLINE DATA CENTER,62714 -,Year,Y,N,SANTA MONICA,LOS ANGELES,CA,90405
167259,I-200-15037-914751,CERTIFIED,2015-02-07,2015-02-17,H-1B,08/02/2015,07/31/2018,INFOSYS LIMITED,6100 TENNYSON PARKWAY,SUITE 200,PLANO,TX,75024,UNITED STATES OF AMERICA,NaN,4.694673e+09,NaN,NaN,NaN,NaN,TECHNOLOGY ARCHITECT - US,15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",541511.0,1.0,Y,77917.00,Year,Level III,OES,2014.0,OFLC ONLINE DATA CENTER,78415 -,Year,Y,N,SANTA MONICA,LOS ANGELES,CA,90405
167804,I-200-15038-130388,WITHDRAWN,2015-02-08,2015-02-11,H-1B,08/04/2015,08/02/2018,INFOSYS LIMITED,6100 TENNYSON PARKWAY,SUITE 200,PLANO,TX,75024,UNITED STATES OF AMERICA,NaN,4.694673e+09,NaN,NaN,NaN,NaN,TECHNOLOGY ANALYST - US,1

In [10]:
print(data.CASE_STATUS.unique())

['WITHDRAWN' 'CERTIFIED' 'DENIED' 'CERTIFIED-WITHDRAWN']


In [11]:
print(data.VISA_CLASS.unique())

['H-1B' 'H-1B1 Chile' 'H-1B1 Singapore' 'E-3 Australian']


In [12]:
data.CASE_SUBMITTED.isnull().sum()

0

In [13]:
data.DECISION_DATE.isnull().sum()

0

In [14]:
data['CASE_SUBMITTED'] = data['CASE_SUBMITTED'].dt.strftime('%Y-%m-%d')
data['DECISION_DATE'] = data['DECISION_DATE'].dt.strftime('%Y-%m-%d')

In [15]:
data.EMPLOYER_ADDRESS = (data.EMPLOYER_ADDRESS1.map(str) +" "+ data.EMPLOYER_ADDRESS2.map(str)).str.replace('nan','').str.upper().str.strip()

In [16]:
data.EMPLOYER_ADDRESS

0                                  710 MEDTRONIC PARKWAY NE
1                                  905 ASP AVE NEL BLDG 207
2                                     6021 CONNECTION DRIVE
3                              9510 N. HOUSTON ROSSLYN ROAD
4                             8250 JONES BRANCH DRIVE MSA3E
5                               2885 COUNTRY DRIVE STE. 175
6                                         3401 HILLVIEW AVE
7                             8250 JONES BRANCH DRIVE MSA3E
8                                         3401 HILLVIEW AVE
9                                  65 CRESTRIDGE DRIVE NONE
10                                        3401 HILLVIEW AVE
11                        21 WEST SUPERIOR STREET SUITE 500
12                              1110 ELDEN STREET SUITE 201
13                                        3401 HILLVIEW AVE
14                              388 WASHINGTON ROAD SUITE A
15                                        3401 HILLVIEW AVE
16                                105 CO

In [53]:
data['EMPLOYER_ADDRESS'] = data.EMPLOYER_ADDRESS.apply(utils.emp_address)


In [54]:
data['EMPLOYER_ADDRESS']

0                                  710 MEDTRONIC PARKWAY NE
1                                  905 ASP AVE NEL BLDG 207
2                                     6021 CONNECTION DRIVE
3                               9510 N HOUSTON ROSSLYN ROAD
4                             8250 JONES BRANCH DRIVE MSA3E
5                                2885 COUNTRY DRIVE STE 175
6                                         3401 HILLVIEW AVE
7                             8250 JONES BRANCH DRIVE MSA3E
8                                         3401 HILLVIEW AVE
9                                  65 CRESTRIDGE DRIVE NONE
10                                        3401 HILLVIEW AVE
11                        21 WEST SUPERIOR STREET SUITE 500
12                              1110 ELDEN STREET SUITE 201
13                                        3401 HILLVIEW AVE
14                              388 WASHINGTON ROAD SUITE A
15                                        3401 HILLVIEW AVE
16                                105 CO

In [19]:
data.EMPLOYER_POSTAL_CODE.isnull().sum()

38

In [20]:
data.EMPLOYER_POSTAL_CODE = data['EMPLOYER_POSTAL_CODE'].apply(utils.update_fix_zip)

Not a number:  FRANKLIN
Not a number:  nan
zip code error:  770042
Not a number:  nan
Not a number:  nan
Not a number:  G74 5PA
Not a number:  nan
Not a number:  nan
Not a number:  nan
Not a number:  nan
Not a number:  nan
Not a number:  ESSEX
Not a number:  V6J 1C7
Not a number:  V6J 1C7
zip code error:  695581
Not a number:  nan
Not a number:  SSAU C0UNTY
Not a number:  nan
Not a number:  nan
Not a number:  nan
zip code error:  541330
Not a number:  MIDDLESEX
zip code error:  002451
zip code error:  966813
zip code error:  695581
Not a number:  nan
Not a number:  nan
Not a number:  PRINCET0N
Not a number:  nan
Not a number:  nan
zip code error:  2345638
zip code error:  2345638
zip code error:  2345638
zip code error:  2345638
Not a number:  PRINCET0N
Not a number:  SAN ANT0NI0
Not a number:  SUNNYVALE
Not a number:  PRINCET0N
Not a number:  C00K
Not a number:  nan
Not a number:  NEW Y0RK
Not a number:  nan
zip code error:  541511
Not a number:  NEW Y0RK
zip code error:  606031
Not a

In [21]:
data.EMPLOYER_POSTAL_CODE.isnull().sum()

172

In [22]:
data.WORKSITE_POSTAL_CODE.isnull().sum()

39

In [23]:
data.WORKSITE_POSTAL_CODE = data['WORKSITE_POSTAL_CODE'].apply(utils.update_fix_zip)

Not a number:  nan
Not a number:  HAMILT0N
zip code error:  200037
Not a number:  nan
zip code error:  194454
zip code error:  941017
Not a number:  M0NTG0MERY
Not a number:  nan
Not a number:  94085AQ05
Not a number:  nan
Not a number:  nan
Not a number:  nan
Not a number:  nan
Not a number:  NEW Y0RK
zip code error:  020215
Not a number:  nan
Not a number:  nan
zip code error:  921203
zip code error:  223314
Not a number:  nan
Not a number:  JERSEY CITY
zip code error:  945898
zip code error:  600018
zip code error:  303051
Not a number:  CHARL0TTE
zip code error:  073020
Not a number:  SAN FRANCISC0
zip code error:  765155
zip code error:  275518
zip code error:  6308600
zip code error:  600018
zip code error:  330338
Not a number:  nan
zip code error:  480176
zip code error:  391111
Not a number:  HUDS0N
zip code error:  432115
zip code error:  7510375886
zip code error:  941407
zip code error:  941407
zip code error:  930003
zip code error:  944020
zip code error:  901367
zip code

In [24]:
data.WORKSITE_POSTAL_CODE.isnull().sum()

672

In [25]:
data["EMPLOYER_CITY"]=data.EMPLOYER_CITY.apply(utils.uppercase_nopunct)
data["EMPLOYER_STATE"]=data.EMPLOYER_STATE.apply(utils.uppercase_nopunct)
data["WORKSITE_CITY"]=data.WORKSITE_CITY.apply(utils.uppercase_nopunct)
data["WORKSITE_STATE"]=data.WORKSITE_STATE.apply(utils.uppercase_nopunct)

In [26]:
data.EMPLOYER_STATE.unique()

array(['MN', 'OK', 'TX', 'VA', 'CA', 'GA', 'NJ', 'NC', 'MI', 'CO', 'PA',
       'WI', 'NY', 'IL', 'CT', 'WA', 'DE', 'AZ', 'MD', 'MO', 'OR', 'IN',
       'KS', 'IA', 'MA', 'TN', 'NE', 'SC', 'LA', 'OH', 'NH', 'RI', 'FL',
       'AR', 'ID', 'AK', 'DC', nan, 'WV', 'KY', 'NM', 'AL', 'NV', 'UT',
       'HI', 'WY', 'VT', 'ND', 'MS', 'MP', 'MT', 'PR', 'SD', 'ME', 'GU',
       'VI'], dtype=object)

In [28]:
data.WORKSITE_STATE = data.WORKSITE_STATE.replace('PW', 'PA')

In [29]:
data.EMPLOYER_STATE.isnull().sum()

43

In [30]:
data.EMPLOYER_STATE = data[['CASE_NUMBER','EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE']].apply(utils.update_fix_states, axis =1)

Old state was nan and new state is NM Case Number: I-200-15064-793184


In [31]:
data.EMPLOYER_STATE.isnull().sum()

42

In [32]:
data.WORKSITE_STATE.isnull().sum()

27

In [33]:
data.WORKSITE_STATE = data[['CASE_NUMBER','WORKSITE_STATE', 'WORKSITE_POSTAL_CODE']].apply(utils.worksite_fix_states, axis =1)

Old state was nan and new state is NY Case Number: I-200-14345-429827
Old state was nan and new state is NY Case Number: I-200-15040-209847
Old state was MH and new state is MA Case Number: I-200-15050-483808
Old state was FM and new state is FL Case Number: I-200-15263-717853


In [34]:
data.WORKSITE_STATE.isnull().sum()

25

In [35]:
data.JOB_TITLE = data['JOB_TITLE'].apply(utils.emp_address)

In [36]:
 data['DOT_CODE'] = None

In [37]:
 data.SOC_CODE = data.SOC_CODE.apply(utils.update_fix_socCode)

SOC code error:  nan | nan
SOC code error:  5-1199.01 | 5-1199.01
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  11-33 | 11-33
SOC code error:  15-2041.2 | 15-2041.2
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  13-111.00 | 13-111.00
SOC code error:  15-112 | 15-112
SOC code error:  15-331 | 15-331
SOC code error:  15-199.02 | 15-199.02
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  27-1021. | 27-1021.
SOC code error:  15-2013.1 | 15-2013.1
SOC code error:  nan | nan
SOC code error:  15-2013.1 | 15-2013.1
SOC code error:  15-199.02 | 15-199.02
SOC code error:  13-161 | 13-161
SOC code error:  nan | nan
SOC code error:  15-1199302 | 15-1199302
SOC code error:  15-119.09 | 15-119.09
SOC code error:  COMPUTER SYSTEMS ANALYSTS | COMPUTERSYSTEMSANALYSTS
SOC co

In [38]:
data.SOC_CODE.unique()

array(['17-2141.00', '25-1032', '15-1799', '17-2072', '15-1131',
       '19-3021.00', '15-1132', '15-2031', '15-1121', '13-2011',
       '15-1021.00', '15-1199', '11-3021', '11-9051.00', '15-1141',
       '17-2071', '15-1051.00', '29-1122', '27-4032', '15-1142',
       '17-2071.00', '17-2051.00', '15-2041.00', '13-2051', '17-2081.00',
       '19-1029', '15-2021', '27-3031', '19-3011.00', '15-2041', '17-2141',
       '13-1111.00', '15-1031.00', '15-1099.01', '17-2031', '13-1199',
       '27-1025', '13-1081.00', '17-2051', '19-4021', '15-2031.00',
       '17-2021', '19-4092.00', '41-1012', '17-2072.00', '11-3031',
       '17-2081', '15-1032.00', '17-2171', '19-2032', '19-2042', '17-1011',
       '15-1134', '13-1161', '19-1099.99', '11-3071', '27-1024.00',
       '13-1151', '27-1024', '21-1023', '17-2131', '29-1123', '19-4041',
       '49-9041.00', '19-1013', '17-2041', '15-1071.00', '19-1021',
       '17-2111.01', '15-1143', '13-2051.00', '25-1011.00', '15-1061.00',
       '19-2031', '21

In [39]:
data.TOTAL_WORKERS.isnull().sum()

15

In [57]:
data.EMPLOYER_PHONE = data.EMPLOYER_PHONE.apply(utils.emp_address)
data.EMPLOYER_PHONE

0         76350527100
1         40532518260
2         97289450000
3         71384917000
4         70391888380
5         33155191110
6         51281331940
7         70391888380
8         51281331940
9         67854102340
10        51281331940
11        21827922440
12        70376688000
13        51281331940
14        73223821220
15        51281331940
16        33642461420
17        70391888380
18        71338680160
19        71338671980
20        62644060330
21        24881320000
22        84786710620
23        73254920300
24        41748339110
25        60882778840
26        70391888380
27        70391888380
28        31525503700
29        80081890780
30        97354283260
31        24852402220
32        40872150000
33        97280199700
34        73270731120
35        21227771000
36        83222012050
37        21827922440
38        61830337530
39        92584386960
40        91927280520
41        56133867370
42        51281331940
43        91946829570
44        51281331940
45        

In [60]:
data['EMPLOYER_AREA_CODE'] = [
    str(i)[:3] if len(str(i)) == 11 else np.nan for i in data['EMPLOYER_PHONE']
]
data.EMPLOYER_AREA_CODE

0         763
1         405
2         972
3         713
4         703
5         331
6         512
7         703
8         512
9         678
10        512
11        218
12        703
13        512
14        732
15        512
16        336
17        703
18        713
19        713
20        626
21        248
22        847
23        732
24        417
25        608
26        703
27        703
28        315
29        800
30        973
31        248
32        408
33        972
34        732
35        212
36        832
37        218
38        618
39        925
40        919
41        561
42        512
43        919
44        512
45        509
46        626
47        915
48        703
49        847
50        956
51        972
52        405
53        734
54        302
55        703
56        703
57        480
58        919
59        480
60        512
61        847
62        847
63        847
64        773
65        703
66        703
67        281
68        713
69        917
70        713
71    

In [42]:
data['WAGE_UNIT_OF_PAY'].unique()

array(['Year', 'Hour', 'Month', nan, 'Week', 'Bi-Weekly'], dtype=object)

In [43]:
data['WAGE_UNIT_OF_PAY'] = data['WAGE_UNIT_OF_PAY'].apply(utils.fix_unitOfPay)

In [44]:
data['WAGE_UNIT_OF_PAY'].unique()

array(['Y', 'H', 'M', None, 'W', 'B'], dtype=object)

In [45]:
data['PW_UNIT_OF_PAY'].unique()

array(['Year', 'Hour', 'Month', nan, 'Week', 'Bi-Weekly'], dtype=object)

In [46]:
data['PW_UNIT_OF_PAY'] = data['PW_UNIT_OF_PAY'].apply(utils.fix_unitOfPay)

In [47]:
data['PW_UNIT_OF_PAY'].unique()

array(['Y', 'H', 'M', None, 'W', 'B'], dtype=object)

In [ ]:
def calc_wage(x):
    wf = x.WAGE_RATE_OF_PAY_FROM
    wt = x.WAGE_RATE_OF_PAY_TO
    if pd.isnull(wt):
        return wf

    if wf <=wt:
        return (wf + wt)/2
    else:
        return wf

In [48]:
a = data['WAGE_RATE_OF_PAY'].map(str).str.split("-").tolist()
a.remove(['22.69', '61.55 ', ' 46.02'])
b = pd.DataFrame(a,columns="WAGE_RATE_OF_PAY_FROM WAGE_RATE_OF_PAY_TO".split())
b['WAGE_RATE_OF_PAY_FROM'] = b['WAGE_RATE_OF_PAY_FROM'].str.strip()
b['WAGE_RATE_OF_PAY_TO'] = b['WAGE_RATE_OF_PAY_TO'].str.strip()
b = b.apply(pd.to_numeric,errors='coerce')

In [49]:
b['WAGE_RATE_OF_PAY_FROM'] = b['WAGE_RATE_OF_PAY_FROM'].replace(0, np.nan)
b['WAGE_RATE_OF_PAY_TO'] = b['WAGE_RATE_OF_PAY_TO'].replace(0, np.nan)
data['WAGE_RATE_OF_PAY'] = b[['WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO']].apply(utils.calc_wage, axis =1)

In [52]:
data.dtypes

CASE_NUMBER               object
CASE_STATUS               object
CASE_SUBMITTED            object
DECISION_DATE             object
VISA_CLASS                object
EMPLOYMENT_START_DATE     object
EMPLOYMENT_END_DATE       object
EMPLOYER_NAME             object
EMPLOYER_ADDRESS1         object
EMPLOYER_ADDRESS2         object
EMPLOYER_CITY             object
EMPLOYER_STATE            object
EMPLOYER_POSTAL_CODE      object
EMPLOYER_COUNTRY          object
EMPLOYER_PROVINCE         object
EMPLOYER_PHONE            object
EMPLOYER_PHONE_EXT       float64
AGENT_ATTORNEY_NAME       object
AGENT_ATTORNEY_CITY       object
AGENT_ATTORNEY_STATE      object
JOB_TITLE                 object
SOC_CODE                  object
SOC_NAME                  object
NAIC_CODE                float64
TOTAL_WORKERS            float64
FULL_TIME_POSITION        object
PREVAILING_WAGE          float64
PW_UNIT_OF_PAY            object
PW_WAGE_LEVEL             object
PW_WAGE_SOURCE            object
PW_WAGE_SO

In [61]:
#Select only the columns that your data has that are in the master list
cdata = data[['CASE_SUBMITTED', 'CASE_NUMBER', 'CASE_STATUS', 'DECISION_DATE', 'VISA_CLASS',
    'JOB_TITLE', 'DOT_CODE', 'SOC_CODE', 'SOC_NAME', 'FULL_TIME_POSITION',
    'EMPLOYER_NAME', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE',
    'EMPLOYER_AREA_CODE', 'NAIC_CODE', 'TOTAL_WORKERS', 
    'WORKSITE_CITY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 
    'WAGE_RATE_OF_PAY', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY']]

In [62]:
cdata.head()

,CASE_SUBMITTED,CASE_NUMBER,CASE_STATUS,DECISION_DATE,VISA_CLASS,JOB_TITLE,DOT_CODE,SOC_CODE,SOC_NAME,FULL_TIME_POSITION,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_AREA_CODE,NAIC_CODE,TOTAL_WORKERS,WORKSITE_CITY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,WAGE_RATE_OF_PAY,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,2015-02-05,I-200-09121-701936,WITHDRAWN,2015-02-05,H-1B,MECHANICAL ENGINEER,None,17-2141.00,MECHANICAL ENGINEERS,Y,"MEDTRONIC, INC.",710 MEDTRONIC PARKWAY NE,MINNEAPOLIS,MN,55432,763,334510.0,1.0,EDEN PRAIRIE,CA,55412,20000.0,Y,19000.0,Y
1,2014-12-12,I-200-09146-796321,CERTIFIED,2014-12-18,H-1B,ASSISTANT PROFESSOR,None,25-1032,"ENGINEERING TEACHERS, POSTSECONDARY",Y,UNIVERSITY OF OKLAHOMA,905 ASP AVE NEL BLDG 207,NORMAN,OK,73019,405,611310.0,1.0,NORMAN,OK,73019,85000.0,Y,42860.0,Y
2,2013-01-03,I-200-09180-329758,WITHDRAWN,2014-12-15,H-1B,RELEASE AND SOFTWARE TEST MANAGER,None,15-1799,"COMPUTER OCCUPATIONS, ALL OTHER*",Y,NOKIA INC.,6021 CONNECTION DRIVE,IRVING,TX,75039,972,517212.0,1.0,SAN DIEGO,CA,92127,94000.0,Y,73965.0,Y
3,2015-03-10,I-200-09183-259985,CERTIFIED,2015-03-16,H-1B,CONTROL ENGINEERING SPECIALIST,None,17-2072,"ELECTRONICS ENGINEERS, EXCEPT COMPUTER",Y,"OMRON OILFIELD AND MARINE, INC.",9510 N HOUSTON ROSSLYN ROAD,HOUSTON,TX,77088,713,335314.0,2.0,HOUSTON,TX,77088,68000.0,Y,65998.0,Y
4,2014-12-17,I-200-09189-800933,CERTIFIED,2014-12-23,H-1B,DEVELOPMENT SENIOR,None,15-1131,COMPUTER PROGRAMMERS,Y,FEDERAL HOME LOAN MORTGAGE CO.,8250 JONES BRANCH DRIVE MSA3E,MCLEAN,VA,22102,703,522294.0,1.0,MCLEAN,VA,22102,120150.0,Y,96907.0,Y


In [63]:
#Write out your clean data to a file

if not os.path.exists('cleaned_data'):
        os.mkdir('cleaned_data')
        
new_filename="2015.csv"
cdata.to_csv("cleaned_data/{}".format(new_filename), encoding='utf-8')